<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/master/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')

In [3]:
# 코랩 실행시
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

In [3]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_train[i] = scaler.fit_transform(df_train[i])
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_test[i] = scaler.fit_transform(df_test[i])

In [5]:
# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

In [6]:
# Delete user_id
df_test = df_test.drop(columns = 'user_id')

# split target
new_x = df_test


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
param_grid = {
    'max_depth': [5,7 ,9, 11],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [ 600,700,800,900,1000],
    'colsample_bytree': [0.3, 0.7, 0.9],
    'num_leaves': [31, 50, 70],
}
# LightGBM 분류기 초기화
lgb_classifier = lgb.LGBMClassifier()

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정s
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=3, scoring=macro_f1_scorer, n_jobs=-1, verbose=1)

# GridSearchCV 실행
grid_search.fit(x_train, y_train)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 Macro F1 점수:", grid_search.best_score_)

# 최적의 모델 가져오기
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(x_test)

# Macro F1 점수 평가
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Test Set Macro F1 Score: {macro_f1}")